In [1]:
import os
import re
import numpy as np
import pandas as pd
import jieba
from gensim.models.word2vec import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

import xgboost as xgb

C:\Program Files\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from sklearn.model_selection import cross_val_score,StratifiedKFold,KFold

In [3]:
def load_dataset(name, nrows=None):
    datasets = {
        'labeled_train': 'train_first.csv',
        'test': 'predict_first.csv'
    }
    if name not in datasets:
        raise ValueError(name)
    data_file = os.path.join('..', 'ccf_data', datasets[name])
    df = pd.read_csv(data_file, sep=',', nrows=nrows)
    print('Number of reviews: {}'.format(len(df)))
    return df

## 2.把训练集和测试集的评论都放进来训练word2vec

In [4]:
df1 = load_dataset('labeled_train').Discuss
df2 = load_dataset('test').Discuss
df_unlabeled = df1.append(df2)
print(df_unlabeled.shape)

Number of reviews: 100000
Number of reviews: 30000
(130000,)


In [71]:
df_unlabeled.head()

0                好大的一个游乐公园，已经去了2次，但感觉还没有玩够似的！会有第三，第四次的
1                      新中国成立也是在这举行，对我们中国人来说有些重要及深刻的意义！
2                  庐山瀑布非常有名，也有非常多个瀑布，只是最好看的非三叠泉莫属，推荐一去
3    个人觉得颐和园是北京最值的一起的地方，不过相比下门票也是最贵的，比起故宫的雄伟与气势磅礴，颐...
4                                               迪斯尼一日游
Name: Discuss, dtype: object

### 1.根据以下这些符号 。！ ? ～  将每条评论分成若干个句子
### 2.然后对每句话进行jieba分词
### 3.最后清洗 保留中文和数字

In [6]:
chn_stopwords = pd.read_csv('../chn_stopwords.csv',encoding='utf-8',header=None)
chn_stopwords.columns = ['c1']
chn_stopwords = set(chn_stopwords.c1)

In [7]:
def clean_text(text, remove_stopwords=False):
    words = list(jieba.cut(text))
    words = [re.sub(r'[^a-z0-9\u4E00-\u9FA5]+','',s1).replace(' ','') for s1 in words]
    words = [s2 for s2 in words if s2]
    if remove_stopwords:
        words = [w for w in words if w not in chn_stopwords]
    return words

In [8]:
def split_sentences(review):
    raw_sentences = re.split(r'。|！|？|～',review.lower().strip())
    sentences = [clean_text(s,remove_stopwords=True) for s in raw_sentences if s]
    return sentences

In [9]:
%%time
sentences = sum(df_unlabeled.apply(split_sentences), [])
print('{} reviews -> {} sentences'.format(len(df_unlabeled), len(sentences)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\16121360\AppData\Local\Temp\jieba.cache
Loading model cost 1.013 seconds.
Prefix dict has been built succesfully.


130000 reviews -> 222159 sentences
Wall time: 5min 7s


## 3.用gensim训练词嵌入模型

In [10]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [11]:
# 设定词向量训练的参数
num_features = 100    # Word vector dimensionality
min_word_count = 7   # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 5          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

model_name = '{}features_{}minwords_{}context.model'.format(num_features, min_word_count, context)

In [12]:
print('Training model...')
model = Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model.save(os.path.join('..', 'ccf_models', model_name))

2018-02-12 16:24:24,423 : INFO : collecting all words and their counts
2018-02-12 16:24:24,424 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-02-12 16:24:24,452 : INFO : PROGRESS: at sentence #10000, processed 84940 words, keeping 18568 word types
2018-02-12 16:24:24,484 : INFO : PROGRESS: at sentence #20000, processed 172449 words, keeping 28614 word types
2018-02-12 16:24:24,510 : INFO : PROGRESS: at sentence #30000, processed 259086 words, keeping 35867 word types
2018-02-12 16:24:24,539 : INFO : PROGRESS: at sentence #40000, processed 346097 words, keeping 41836 word types
2018-02-12 16:24:24,569 : INFO : PROGRESS: at sentence #50000, processed 430042 words, keeping 46786 word types
2018-02-12 16:24:24,601 : INFO : PROGRESS: at sentence #60000, processed 517171 words, keeping 51793 word types


Training model...


2018-02-12 16:24:24,630 : INFO : PROGRESS: at sentence #70000, processed 603607 words, keeping 56188 word types
2018-02-12 16:24:24,657 : INFO : PROGRESS: at sentence #80000, processed 688055 words, keeping 60049 word types
2018-02-12 16:24:24,692 : INFO : PROGRESS: at sentence #90000, processed 773379 words, keeping 63733 word types
2018-02-12 16:24:24,725 : INFO : PROGRESS: at sentence #100000, processed 860854 words, keeping 67186 word types
2018-02-12 16:24:24,750 : INFO : PROGRESS: at sentence #110000, processed 948239 words, keeping 70515 word types
2018-02-12 16:24:24,780 : INFO : PROGRESS: at sentence #120000, processed 1037059 words, keeping 73725 word types
2018-02-12 16:24:24,809 : INFO : PROGRESS: at sentence #130000, processed 1122157 words, keeping 76337 word types
2018-02-12 16:24:24,841 : INFO : PROGRESS: at sentence #140000, processed 1208520 words, keeping 78807 word types
2018-02-12 16:24:24,869 : INFO : PROGRESS: at sentence #150000, processed 1295839 words, keeping

In [72]:
# 测试下similarity
model.similar_by_word('不错')

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  from ipykernel import kernelapp as app


[('挺不错', 0.8409960269927979),
 ('很棒', 0.760872483253479),
 ('好', 0.7602142095565796),
 ('算', 0.720684289932251),
 ('更好', 0.7083621025085449),
 ('真不错', 0.6983656287193298),
 ('还行', 0.6846714615821838),
 ('挺好玩', 0.6446146368980408),
 ('很赞', 0.6442148685455322),
 ('超棒', 0.6314824223518372)]

## 读入之前训练好的Word2Vec模型

In [12]:
model_name = '100features_7minwords_5context.model'
model = Word2Vec.load(os.path.join('..', 'ccf_models', model_name))

2018-02-13 14:25:08,140 : INFO : loading Word2Vec object from ..\ccf_models\100features_7minwords_5context.model
2018-02-13 14:25:08,667 : INFO : loading trainables recursively from ..\ccf_models\100features_7minwords_5context.model.trainables.* with mmap=None
2018-02-13 14:25:08,667 : INFO : loading wv recursively from ..\ccf_models\100features_7minwords_5context.model.wv.* with mmap=None
2018-02-13 14:25:08,667 : INFO : setting ignored attribute vectors_norm to None
2018-02-13 14:25:08,667 : INFO : loading vocabulary recursively from ..\ccf_models\100features_7minwords_5context.model.vocabulary.* with mmap=None
2018-02-13 14:25:08,667 : INFO : setting ignored attribute cum_table to None
C:\Program Files\Anaconda3\lib\site-packages\gensim\utils.py:461: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  setattr(self, attrib, None)
2018-02-13 14:25:08,667 : INFO : loaded ..\ccf_models\100features_7minwords_5c

### 我们可以根据word2vec的结果去对评论进行编码

In [14]:
# array.mean相当于把100维度取了平均
def to_review_vector(review):
    words = clean_text(review, remove_stopwords=True)
    array = np.array([model[w] for w in words if w in model])
    return pd.Series(array.mean(axis=0))

In [15]:
df = load_dataset('labeled_train')
df.head()

Number of reviews: 100000


Id  \
0  201e8bf2-77a2-3a98-9fcf-4ce03914e712   
1  f4d51947-eac4-3005-9d3c-2f32d6068a2d   
2  74aa7ae4-03a4-394c-bee0-5702d3a3082a   
3  099661c2-4360-3c49-a2fe-8c783764f7db   
4  97ca672d-e558-3542-ba7b-ee719bba1bab   

                                             Discuss  Score  
0              好大的一个游乐公园，已经去了2次，但感觉还没有玩够似的！会有第三，第四次的      5  
1                    新中国成立也是在这举行，对我们中国人来说有些重要及深刻的意义！      4  
2                庐山瀑布非常有名，也有非常多个瀑布，只是最好看的非三叠泉莫属，推荐一去      4  
3  个人觉得颐和园是北京最值的一起的地方，不过相比下门票也是最贵的，比起故宫的雄伟与气势磅礴，颐...      5  
4                                             迪斯尼一日游      5

In [18]:
train_data_features = df.Discuss.apply(to_review_vector)
train_data_features.head()

## 4.用随机森林构建分类器

In [19]:
# 去掉空值行
train_data_features = train_data_features.fillna(0)

## 4.1进行交叉验证

In [47]:
# 自己写评判函数
from sklearn.metrics import scorer
def ff(y,y_pred):
    rmse = np.sqrt(sum((y-y_pred)**2)/len(y))
    score = 1.0 / (1+rmse)
    return score
rmse_scoring = scorer.make_scorer(ff)

In [51]:
# 选出最佳参数 n_estimators=300, max_depth=8
forest = RandomForestClassifier(n_estimators = 300,max_depth=8, random_state=42,n_jobs=4)
cross_val_score(forest,X=train_data_features,y=df.Score,scoring=rmse_scoring,cv=3)

array([ 0.55202775,  0.55153875,  0.55163238])

In [59]:
# 拟合模型
forest = RandomForestClassifier(n_estimators = 300,max_depth=8, random_state=42,n_jobs=4)
forest = forest.fit(train_data_features, df.Score)

## 5.预测

In [100]:
df = load_dataset('test')
df.head()

Number of reviews: 30000


Id                           Discuss
0  9a1caf96-681e-3c11-b588-43ac742d7fd2                              快乐之旅
1  82b450db-65c2-351c-84fb-761d76582680               岛上看日落的地方，视野很开阔，非常漂亮
2  2eec4606-590c-3fa2-b846-7f92441c54a6                很有鲁迅风味 很喜欢这样有文化的地方
3  509f9a68-ac41-35ff-9d2e-2fc12f73ed7f               去乌鲁木齐还能不去天山天池吗，哈哈哈～
4  395f4b22-1c5f-328a-a19d-5065e0530cbc  非常满意，直接拿身份证刷机入园就行了，不用排队买票，比较节约时间

In [62]:
test_data_features = df.Discuss.apply(to_review_vector)
test_data_features.head()

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: RuntimeWarning: Mean of empty slice.


0         1         2         3         4         5         6   \
0 -0.057594 -0.106595 -0.011975 -0.037539 -0.019648  0.064127  0.015033   
1  0.028080  0.051105  0.033921 -0.047737 -0.017698 -0.034105  0.046771   
2 -0.089150  0.030696 -0.116034 -0.031124  0.004997  0.072179  0.046005   
3  0.068896 -0.072747  0.039307 -0.028288  0.027445 -0.079145  0.073709   
4  0.027088  0.090219  0.010050  0.087851 -0.093998 -0.041130  0.028349   

         7         8         9     ...           90        91        92  \
0  0.107385 -0.039085 -0.040876    ...    -0.082075 -0.023566  0.038449   
1  0.048276  0.047799  0.053685    ...    -0.143990 -0.058739 -0.001297   
2 -0.017897 -0.001946  0.106562    ...     0.002467  0.024220 -0.090781   
3  0.101741  0.044884  0.056739    ...    -0.134339 -0.021579  0.012122   
4  0.006541  0.153597  0.095631    ...    -0.124617 -0.005774  0.032768   

         93        94        95        96        97        98        99  
0 -0.083291 -0.011352  0.106291 -0.142022  0.064086 -0.266560  0.105870  
1  0.069079 -0.014151 -0.030022 -0.007411 -0.140884 -0.005071 -0.018426  
2  0.023268 -0.077763  0.006088 -0.118615 -0.027255 -0.146589  0.057561  
3 -0.045251 -0.019853  0.030651 -0.016670  0.104854  0.045399 -0.013049  
4  0.038773  0.028960  0.157871  0.108495  0.123693  0.070204  0.081022  

[5 rows x 100 columns]

In [63]:
test_data_features = test_data_features.fillna(0)

In [64]:
result = forest.predict(test_data_features)
output = pd.DataFrame({'Id':df.Id, 'Score':result})

In [68]:
# save
output.to_csv(os.path.join('..', 'ccf_data', 'submit.csv'),sep=',',index=False,header=False)
output.head()

Id  Score
0  9a1caf96-681e-3c11-b588-43ac742d7fd2      5
1  82b450db-65c2-351c-84fb-761d76582680      5
2  2eec4606-590c-3fa2-b846-7f92441c54a6      5
3  509f9a68-ac41-35ff-9d2e-2fc12f73ed7f      5
4  395f4b22-1c5f-328a-a19d-5065e0530cbc      5

In [ ]:
##############################################

In [81]:
# 将多分类变成二分类看看 4和5标记为1，其他为0
df.Score.value_counts()

5    60097
4    28954
3     9389
2      973
1      587
Name: Score, dtype: int64

In [92]:
a = pd.Series(np.where(df.Score > 3, 1, 0), name='result')

In [93]:
a.value_counts()

1    89051
0    10949
Name: result, dtype: int64

In [110]:
# 选出最佳参数 n_estimators=?, max_depth=?
forest = RandomForestClassifier(n_estimators = 200,max_depth=9, random_state=42,n_jobs=4)
cross_val_score(forest,X=train_data_features,y=a,scoring='roc_auc',cv=3)

array([ 0.7577724 ,  0.75761179,  0.76362349])

In [111]:
# 拟合 0 1
# 预测
forest = forest.fit(train_data_features, a)
result = forest.predict(test_data_features)
output = pd.DataFrame({'Id':df.Id, 'Score':result})

In [112]:
output.Score.value_counts()

1    29903
0       97
Name: Score, dtype: int64

In [115]:
# 将0和1替换成2和4
b = pd.Series(np.where(output.Score == 1, 4, 2), name='Score')
output['Score'] = b

In [117]:
# save
output.to_csv(os.path.join('..', 'ccf_data', 'submit3.csv'),sep=',',index=False,header=False)
output.head()

Id  Score
0  9a1caf96-681e-3c11-b588-43ac742d7fd2      4
1  82b450db-65c2-351c-84fb-761d76582680      4
2  2eec4606-590c-3fa2-b846-7f92441c54a6      4
3  509f9a68-ac41-35ff-9d2e-2fc12f73ed7f      4
4  395f4b22-1c5f-328a-a19d-5065e0530cbc      4